# Create a Dummy Model

This notebook creates a simple, compatible TensorFlow Lite HAR model (11 activities).
This version uses basic layers that are fully supported by TensorFlow Lite.

In [12]:
import json
import numpy as np
from pathlib import Path
import tensorflow as tf

In [15]:
# --- Activity labels (index order matches the softmax outputs) ---
CLASS_NAMES = [
    "Standing",
    "Lying down on left",
    "Lying down right",
    "Lying down back",
    "Lying down on stomach",
    "Normal walking",
    "Ascending stairs",
    "Descending stairs",
    "Shuffle walking",
    "Running",
    "Miscellaneous movements",
]

def create_minimal_tflite_model():
    """
    Creates a minimal TensorFlow Lite model, even if TensorFlow isn't installed.
    - If TensorFlow is available, builds a real model.
    - Otherwise, creates a small mock model for testing purposes.
    """
    try:
        # Try to import TensorFlow. If available, make a proper TFLite model.
        import tensorflow as tf
        print("TensorFlow found! Creating optimised model...")
        return create_tensorflow_model()

    except ImportError:
        # Fallback path if TensorFlow isn’t installed.
        print("TensorFlow not found. Creating minimal compatible model...")

        # Create a dummy .tflite file so the app can still run tests.
        model_path = create_mock_tflite_file()

        # Save class labels next to the model file (e.g., labels.txt).
        save_labels_alongside_model(model_path, CLASS_NAMES)

        # Return the mock model’s path.
        return model_path



def create_tensorflow_model():
    """Creates and saves a simple TensorFlow Lite model for activity recognition."""

    # --- Model configuration ---
    window_size = 128          # Number of timesteps per input window
    n_features = 3             # e.g., accelerometer axes: x, y, z
    n_classes = len(CLASS_NAMES)  # Number of activity labels

    print(f"Creating model with input shape: ({window_size}, {n_features})")
    print(f"Output classes: {n_classes} -> {CLASS_NAMES}")

    # --- Define a minimal, TFLite-friendly Keras model ---
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(window_size, n_features), name='input'),
        tf.keras.layers.Flatten(),                          # Simplest way to flatten sequence data
        tf.keras.layers.Dense(32, activation='relu', name='dense1'),
        tf.keras.layers.Dense(16, activation='relu', name='dense2'),
        tf.keras.layers.Dense(n_classes, activation='softmax', name='output')
    ])

    # Compile using common, widely supported settings
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("Model architecture:")
    model.summary()

    # --- Generate dummy data for quick training ---
    print("Generating training data...")
    X = np.random.randn(600, window_size, n_features).astype(np.float32)  # random input samples
    y = tf.keras.utils.to_categorical(
        np.random.randint(0, n_classes, 600), num_classes=n_classes       # random labels
    )

    # Briefly train to initialize weights (not for real accuracy)
    print("Training model briefly...")
    model.fit(X, y, epochs=3, batch_size=32, verbose=1, validation_split=0.2)

    # --- Convert model to TensorFlow Lite format ---
    print("Converting to TensorFlow Lite...")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = []                        # Keep conversion simple
    converter.target_spec.supported_types = [tf.float32] # Force float32 for compatibility
    tflite_model = converter.convert()

    # --- Save the converted model ---
    model_path = "har_model.tflite"
    with open(model_path, 'wb') as f:
        f.write(tflite_model)

    print(f"✓ Compatible TensorFlow Lite model saved: {model_path}")
    print(f"Model size: {len(tflite_model) / 1024:.1f} KB")

    # --- Save labels and test the model ---
    save_labels_alongside_model(model_path, CLASS_NAMES)
    test_tflite_model(model_path, X[:3])  # Optional sanity check

    return model_path


def create_mock_tflite_file():
    """
    Creates a minimal dummy .tflite file for testing when TensorFlow isn't available.
    - This file mimics the structure of a TensorFlow Lite model just enough
      for apps to test file handling or loading logic.
    - It will NOT run inference.
    """
    print("Creating minimal test file...")

    # Start with the TensorFlow Lite flatbuffer magic number "TFL3"
    # (all .tflite files begin with this 4-byte header)
    mock_content = b"TFL3"

    # Add some arbitrary padding bytes to make the file non-empty.
    # 100 bytes is enough to simulate a small model file.
    mock_content += b"\x00" * 100

    # Define where to save the mock model
    model_path = "har_model_mock.tflite"

    # Write the mock content to disk
    with open(model_path, 'wb') as f:
        f.write(mock_content)

    print(f"✓ Mock model file created: {model_path}")
    print("⚠️  This is a test file only — install TensorFlow for a real model")

    return model_path



def save_labels_alongside_model(model_path: str, labels):
    """
    Saves the provided class labels next to the model file.
    Creates both:
      - labels.txt  → one label per line (simple and human-readable)
      - labels.json → structured format for programmatic access
    """
    # Get the directory where the model file is located
    p = Path(model_path).resolve().parent

    # Define paths for both label files
    txt_path = p / "labels.txt"
    json_path = p / "labels.json"

    # --- Save plain text labels (useful for quick debugging or mobile apps) ---
    with open(txt_path, "w", encoding="utf-8") as f:
        for name in labels:
            f.write(name + "\n")

    # --- Save JSON labels (useful for structured access in code) ---
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump({"class_names": labels}, f, ensure_ascii=False, indent=2)

    print(f"✓ Labels saved: {txt_path.name}, {json_path.name}")




def test_tflite_model(model_path, test_samples):
    """
    Performs a quick test to verify that a TensorFlow Lite model can load
    and run inference successfully.

    Args:
        model_path (str): Path to the .tflite file to test.
        test_samples (np.ndarray): Example input samples to run through the model.

    Returns:
        bool: True if the model loads and runs correctly, False otherwise.
    """
    try:

        print(f"Testing model: {model_path}")

        # --- Load and initialize the TFLite interpreter ---
        interpreter = tf.lite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()  # prepares model for inference

        # Retrieve model input/output info
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        print(f"✓ Input shape: {input_details[0]['shape']}")
        print(f"✓ Output shape: {output_details[0]['shape']} (n_classes={len(CLASS_NAMES)})")

        # --- Run a couple of inference tests ---
        for i, sample in enumerate(test_samples[:2]):  # test first 2 samples
            input_data = np.array([sample], dtype=np.float32)

            # Feed data to the model
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()  # run inference

            # Retrieve prediction results
            output_data = interpreter.get_tensor(output_details[0]['index'])

            predicted_class = int(np.argmax(output_data[0]))
            confidence = float(output_data[0][predicted_class])

            print(f"Sample {i+1}: Class {predicted_class} - "
                  f"{CLASS_NAMES[predicted_class]} ({confidence:.3f})")

        print("✓ Model test successful!")
        return True

    except Exception as e:
        # Catch all errors (e.g., missing TensorFlow, bad model file, etc.)
        print(f"Model test failed: {e}")
        return False



In [16]:
print("TensorFlow Lite HAR Model Creator (Compatible Version, 11 classes)")
print("=" * 70)

model_path = create_minimal_tflite_model()

print(f"\n🎉 Model created: {model_path}")
print("\n📋 Next steps:")
print("1. Copy these files to your Flutter app assets:")
print("   har_model.tflite")
print("2. Update pubspec.yaml to include both files and restart the app.")
print("3. The app should now show 'HAR Model Ready' and map outputs to labels.")

TensorFlow Lite HAR Model Creator (Compatible Version, 11 classes)
TensorFlow found! Creating optimised model...
Creating model with input shape: (128, 3)
Output classes: 11 -> ['Standing', 'Lying down on left', 'Lying down right', 'Lying down back', 'Lying down on stomach', 'Normal walking', 'Ascending stairs', 'Descending stairs', 'Shuffle walking', 'Running', 'Miscellaneous movements']
Model architecture:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_5 (Flatten)             │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 32)             │        12,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 11)             │           187 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,035 (50.92 KB)

 Trainable params: 13,035 (50.92 KB)

 Non-trainable params: 0 (0.00 B)

Generating training data...
Training model briefly...
Epoch 1/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.0849 - loss: 2.6534 - val_accuracy: 0.0667 - val_loss: 2.6059
Epoch 2/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1699 - loss: 2.3787 - val_accuracy: 0.0917 - val_loss: 2.5578
Epoch 3/3
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1783 - loss: 2.2246 - val_accuracy: 0.0917 - val_loss: 2.5382
Converting to TensorFlow Lite...
Saved artifact at '/tmp/tmphuipnwvg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 11), dtype=tf.float32, name=None)
Captures:
  140346685416336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140346685417488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140346685415952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140346685417680: TensorSpec(shape=(), dtype=tf.resou

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
